<h1 style = "text-align: center"> 
    <font color ='green'> <b>IMDb Movie data using python web-scrapping </b> </font></h1>
<h5 style = "text-align: left"> 
    <font color ='green'> <i> <b>Author: D.Sathyakala,</b></i></font></h5>
<h5 style = "text-align: left"> 
    <font color ='green'> <i><b>Data analyst Intern @ Senchola Technology Solutions</b></i> </font></h5>


<h2 style = "text-align: center"> 
    <font color ='blue'> Web-scrapping </font></h2>

<h3 style = "text-align: left"> 
    <font color ='red'> <i>Import library files</i>  </font></h3>

In [19]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re

<h3 style = "text-align: left"> 
    <font color ='red'> <i>Steps</i>  </font></h3>

In [20]:
try:
    # loading url of top 250 movies of IMBb list
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    
    # url of the IMDb website
    url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
    
    # Get the url from requests library
    response = requests.get(url,headers = {'User-Agent':user_agent,'Accept-Language' : 'en-US,en;q=0.5'})
    print(response)
    
    # Souping and arranging the html code
    soup = bs(response.text,'html.parser')
    
    # Select table from the soup
    movies = soup.findAll('div', attrs = {'class':'sc-c7e5f54-0 gytZrF cli-children'})
    #print(movies)

    # Select the required coulmn for the dataframe
    movie_name = []
    year = []
    time = []
    rating = []
    rating_type = []
    votes = []

    # Taking each records and building our dataframe
    for source in movies:
        # Get movie name 
        name = source.div.a.text
        movie_name.append(name)
    
        # Get year of release
        year1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').text
        year.append(year1)
    
        # Get duration of movie
        time1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').find_next('span').text
        time.append(time1)
    
        # Get rating type of movie
        rating_type1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').find_next('span').find_next('span').text
        rating_type.append(rating_type1)
    
        # Get voting rates of movie
        vote1 = source.find('span',class_="ipc-rating-star--voteCount").text.replace('\xa0(','').replace(')','')
        votes.append(vote1)
    
        # Get the rating of movie
        rating1 = source.find('div',class_="sc-e3e7b191-0 iKUUVe sc-c7e5f54-2 hCiLPi cli-ratings-container").text.split('(')[0].replace('\xa0','')
        rating.append(rating1)
except Exception as e:
    print(e)


<Response [504]>


In [21]:
# Put all details into the Dataframe
df = pd.DataFrame({'Movie Name':movie_name, 'Duration':time, 'Released year':year, 'Rating': rating, 'Rated type':rating_type, 'Votes':votes})
df


,Movie Name,Duration,Released year,Rating,Rated type,Votes


    Here I will modify the column Movie Name, Duration, Votes in proper way  

In [22]:
# Convert duration (hours) into minutes
def convert_to_minutes(runtime):
    total_minutes = 0

    parts = runtime.split()
    for part in parts:
        if 'h' in part:
            total_minutes += int(part.replace('h','')) * 60
        elif 'm' in part:
            total_minutes += int(part.replace('m',''))
    return total_minutes

In [23]:
# Convert viewers count (Mega, Kilo) into whole value
def convert_viewer_count(viewer_count_str):
     multiplier = 1
     if 'M' in viewer_count_str:
         multiplier = 1e6
     elif 'K' in viewer_count_str:
         multiplier = 1e3
     return int(float(viewer_count_str.replace('M', '').replace('K', '')) * multiplier)


In [26]:
try:
    # loading url of top 250 movies of IMBb list
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    
    # url of the IMDb website
    url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
    
    # Get the url from requests library
    response = requests.get(url,headers = {'User-Agent':user_agent,'Accept-Language' : 'en-US,en;q=0.5'})
    print(response)
    
    # Souping and arranging the html code
    soup = bs(response.text,'html.parser')
    
    # Select table from the soup
    movies = soup.findAll('div', attrs = {'class':'sc-c7e5f54-0 gytZrF cli-children'})
    #print(movies)

    # Select the required coulmn for the dataframe
    rank = []
    movie_name = []
    year = []
    time = []
    rating = []
    rating_type = []
    votes = []
        
    # Taking each records and building our dataframe
    for source in movies:
        name = source.div.a.text

        # Get the rank of the movie
        rank1 = int(name.split('. ')[0])
        
        # Get name of the movie
        name1 = name.split('. ')[1]
        rank.append(rank1)
        movie_name.append(name1)
        
        # Get year of release
        year1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').text
        year.append(year1)
        
        # Get duration of movie in minutes
        time1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').find_next('span').text
        time1 = convert_to_minutes(time1)
        time.append(time1)
        
        # Get rating type of movie
        rating_type1 = source.find('div',class_="sc-c7e5f54-7 brlapf cli-title-metadata").find('span').find_next('span').find_next('span').text
        rating_type.append(rating_type1)
        
         # Get voting rates of movie in whole number
        vote1 = source.find('span',class_="ipc-rating-star--voteCount").text.replace('\xa0(','').replace(')','')
        vote1 = convert_viewer_count(vote1)
        votes.append(vote1)
        
        # Get the rating of movie
        rating1 = source.find('div',class_="sc-e3e7b191-0 iKUUVe sc-c7e5f54-2 hCiLPi cli-ratings-container").text.split('(')[0].replace('\xa0','')
        rating.append(rating1)

except Exception as e:
    print(e)

<Response [200]>


In [27]:
# Put all details into the Dataframe
df1 = pd.DataFrame({'Rank':rank, 'Movie Name':movie_name, 'Year':year, 'Duration':time, 'Rating':rating, 'Rating type':rating_type, 'Vote':votes})
df1


,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote


In [28]:
# Dounload the dataFrame into csv file
df1.to_csv('IMDb dataset.csv',index = False, encoding = 'latin')


<h2 style = "text-align: center"> 
    <font color ='blue'> Data Exploration  </font></h2>

In [29]:
# Load the dataset
data = pd.read_csv("IMDb dataset.csv",encoding="latin")
data

,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote


In [30]:
# Display all the details of dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rank         0 non-null      object
 1   Movie Name   0 non-null      object
 2   Year         0 non-null      object
 3   Duration     0 non-null      object
 4   Rating       0 non-null      object
 5   Rating type  0 non-null      object
 6   Vote         0 non-null      object
dtypes: object(7)
memory usage: 132.0+ bytes


In [31]:
# Display top 5 rows of the dataset
data.head(5)

,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote


In [32]:
# Display bottom 5 rows of the dataset
data.tail(5)

,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote


In [33]:
# Display size of the dataset
data.shape

(0, 7)

In [34]:
# Display the statistics of the dataset
data.describe()

,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote
count,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Display column name of the dataset
data.columns

Index(['Rank', 'Movie Name', 'Year', 'Duration', 'Rating', 'Rating type',
       'Vote'],
      dtype='object')

In [36]:
# Check the duplicate data
data.duplicated().sum()

0

In [37]:
# Identify the null values
data.isna()

,Rank,Movie Name,Year,Duration,Rating,Rating type,Vote


In [38]:
# Identify the null values
data.isna().sum()

Rank           0.0
Movie Name     0.0
Year           0.0
Duration       0.0
Rating         0.0
Rating type    0.0
Vote           0.0
dtype: float64